In [1]:
import os
import sys
import json
import random
import pandas as pd
import numpy as np
from collections import deque
from src.model import BiLSTM_CRF, MakeEmbed, textCNN, EpochLogger, save
from src.dataset import Preprocessing, MakeDataset
from src.NLU import NaturalLanguageUnderstanding
from src.DM import DialogManager, E2E_dialog

In [4]:
class DialogSystem:
    def __init__(self):
        entity_pretrain_path = './chatbot_data/pretraining/1_entity_recog_model/entity_recog_20.89_steps_2.pt'
        intent_pretrain_path = './chatbot_data/pretraining/1_intent_clsf_model/intent_clsf_99.585_steps_7.pt'
        ood_pretrain_path = './chatbot_data/pretraining/1_ood_clsf_model/ood_clsf_99.663_steps_4.pt'
        chitchat_pretrain_path = './chatbot_data/pretraining/4_chitchat_trasnformer_model/chitchat_transformer_0.035401_steps_69.pt'
        
        self.NLU = NaturalLanguageUnderstanding()
        
        self.NLU.model_load(intent_pretrain_path, entity_pretrain_path, ood_pretrain_path)
        self.dm = DialogManager()
        
        self.e2e = E2E_dialog(self.NLU.dataset, chitchat_pretrain_path)
        
        self.result = {
            'DM_result' : {},
            'E2E_result' : [],
            'NLG' : "",
        }
        
    def task_oriented_dialog_system(self, q):
        NLU_result = self.NLU.run(q)
        
        system_response = self.dm.run(NLU_result)
        return system_response
    
    def chit_chat_dialog_system(self, q):
        system_response = self.e2e.predict(q)
        return system_response
    
    def ranking(self, dm_result, chitchat_result):
        self.result['DM_result'] = dm_result
        self.result['E2E_result'] = list(chitchat_result)
        dm_nlg = dm_result['NLG']
        
        if(not dm_nlg == []):
            self.result['NLG'] = dm_nlg
            return self.result
        else:
            score = chitchat_result[1]
            print(score)
            if(score>0.05):
                self.result['NLG'] = chitchat_result[0]
                return self.result
            else:
                self.result['NLG'] = 'Sorry, Once again.'
                return self.result
    
    def run(self, q):
        dm_result = self.task_oriented_dialog_system(q)
        chitchat_result = self.chit_chat_dialog_system(q)
        result = self.ranking(dm_result, chitchat_result)
        return result           
            

In [5]:
dialogsystem = DialogSystem()

In [6]:
result = dialogsystem.run('안녕')

C:\Users\ParkHanna\anaconda3\lib\site-packages\torchcrf\__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [7]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': ['LOCATION^안녕'],
  'NLU': {'INTENT': 'ood', 'SLOT': ['LOCATION^안녕']},
  'NLG': ['안녕하세요! 날씨 미세먼지 여행 식당에 대해 물어봐주세요!']},
 'E2E_result': ['안타깝네요.', 0.8144564032554626],
 'NLG': ['안녕하세요! 날씨 미세먼지 여행 식당에 대해 물어봐주세요!']}

In [8]:
result = dialogsystem.run('인천 맛집')

In [9]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': ['LOCATION^인천'],
  'NLU': {'INTENT': 'restaurant', 'SLOT': ['LOCATION^인천']},
  'NLG': ['인천 주변의 추천 식당은 OOOO입니다.']},
 'E2E_result': ['하는데 바로가봐요한 거죠.', 0.4829944968223572],
 'NLG': ['인천 주변의 추천 식당은 OOOO입니다.']}

In [10]:
result =  dialogsystem.run('미세먼지 알려줘')

In [11]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': [],
  'NLU': {'INTENT': 'dust', 'SLOT': []},
  'NLG': ['오늘 서울의 미세먼지는 좋음입니다.']},
 'E2E_result': ['집에 잇몸약 집에 잇몸약 인생의 집에 잇몸약 인생의 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요 마스크 아빠 나가세요.',
  0.635953426361084],
 'NLG': ['오늘 서울의 미세먼지는 좋음입니다.']}

In [12]:
result = dialogsystem.run('대전')

In [14]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': ['LOCATION^대전'],
  'NLU': {'INTENT': 'ood', 'SLOT': ['LOCATION^대전']},
  'NLG': ['오늘 대전의 미세먼지는 좋음입니다.']},
 'E2E_result': ['상대에게 휘둘려선 안 돼요.', 0.4349207580089569],
 'NLG': ['오늘 대전의 미세먼지는 좋음입니다.']}

In [15]:
dialogsystem.run("날씨 알려줘")

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': [],
  'NLU': {'INTENT': 'weather', 'SLOT': []},
  'NLG': ['오늘 서울의 날씨는 흐림입니다.']},
 'E2E_result': ['나쁜놈 가보세요.', 0.5377364754676819],
 'NLG': ['오늘 서울의 날씨는 흐림입니다.']}

In [16]:
result=dialogsystem.run("난 혼자인게 좋아")

0.6368664503097534


In [17]:
result

{'DM_result': {'STATE': '', 'SLOT': '', 'NLU': {}, 'NLG': []},
 'E2E_result': ['자꾸 같이 놀아요.', 0.6368664503097534],
 'NLG': '자꾸 같이 놀아요.'}

In [18]:
result=dialogsystem.run("꽃을 좋아해")

0.40709343552589417


In [19]:
result

{'DM_result': {'STATE': '', 'SLOT': '', 'NLU': {}, 'NLG': []},
 'E2E_result': ['되었나봐요 정리해보세요기 잊지 함께 화장실 대화를 익숙해지지가 아니에요.',
  0.40709343552589417],
 'NLG': '되었나봐요 정리해보세요기 잊지 함께 화장실 대화를 익숙해지지가 아니에요.'}

In [ ]:
dialogsystem.dm.clear()

q = ""
q = input()
while(q!='end'):
    result = dialogsystem.run(q)
    print(result['NLG'])
    q = input()
    
dialogsystem.dm.clear()

 안녕


['안녕하세요! 날씨 미세먼지 여행 식당에 대해 물어봐주세요!']


 오늘 미세먼지 알려줘


['오늘 서울의 미세먼지는 나쁨입니다.']


 대전은?


...

0.4603741466999054
그래도 바라는 다시 만날 거예요.


 그래


0.5119590163230896
다른 귀찮아 꾸준히 돌려보세요.


 알았어


0.6008734703063965
저도 하나니까요.
